In [1]:
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import iplot
import ipywidgets as widgets
from ipywidgets import interact, RadioButtons, IntSlider, Layout
import cufflinks as cf
cf.set_config_file(offline=True)
import plotly.io as pio
pio.renderers.default='notebook'

In [2]:
df = pd.read_csv('https://github.com/Lirinawi/Interactive_chart/blob/main/sales_data.csv?raw=true', 
                 encoding='cp1251', names=['all'])
df.sample(5)

,all
154,06.01 - 3211грн (82) 272п
294,26.05 - 6811грн (93) 426п
133,17.12 - 6911грн (33) 413п 18:20+-
670,27.06 - 4710грн (86) 386п
413,07.10 - 9211грн (30) 690п


In [3]:
df.dropna(inplace=True, how='all')
df['year'] = df['all'].str.extract(r'(\d{4})год').fillna(method="ffill")
df['day_month'] = df['all'].str.extract(r'(\d{2}\.\d{2}).+') 
df['date'] = df['day_month']  + '.' + df['year']
df['profit'] = df['all'].str.extract(r'(\d{2,4})\s?грн')
df['cnt_orders'] = df['all'].str.extract(r'\((\d{2})\)')
df['cnt_positions'] = df['all'].str.extract(r'(\d{2,3})п')
df['days_off'] = df['all'].apply(lambda x: 1 if 'выходной' in x else 0)
df.dropna(subset='date', inplace=True)
df_ = df.iloc[:, 3:]
df_[['profit', 'cnt_orders', 'cnt_positions']] = df_[['profit', 'cnt_orders', 'cnt_positions']].astype('Int32')
df_['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df_2 = df_.set_index('date').fillna(0)
df_2.sample(5)

,profit,cnt_orders,cnt_positions,days_off
date,,,,
2019-04-06,6120,90,490,0
2019-04-16,0,0,0,1
2019-08-30,6411,92,445,0
2022-06-14,6311,86,371,0
2021-11-12,9311,91,524,0


In [4]:
df_report = df_.groupby([df_['date'].dt.year, df_['date'].dt.month]).sum().sort_index()
df_report = df_report.reset_index(level=1).rename(columns={'date': 'month'}).reset_index().rename(
    columns={'date': 'year'})
df_report['year,month'] = df_report['year'].astype('str') + '/' + df_report['month'].astype('str')
df_report_2 = df_report.copy()
df_report_2['year,month'] = pd.to_datetime(df_report_2['year,month'], format='%Y/%m')
df_report_2['year,month'] = df_report_2['year,month'].dt.floor('d')
df_report_2.rename(columns={'year,month':'date'}, inplace=True)
df_report_2.set_index('date', inplace=True)
df_report_2.sample(5)

,year,month,profit,cnt_orders,cnt_positions,days_off
date,,,,,,
2020-01-01,2020,1,109470,1688,9307,4
2018-10-01,2018,10,126698,2162,0,6
2021-11-01,2021,11,150770,2024,11051,8
2019-03-01,2019,3,149016,2140,10074,7
2021-08-01,2021,8,142270,1891,12000,9


In [5]:
analysis_by=RadioButtons(options=['month', 'days', 'None'], value='None', description='analysis_by')
year=IntSlider(min=df_report_2.year.min(), max=df_report_2.year.max(), step=1, value=2022, description='year')
month=IntSlider(min=1, max=12, step=1, value=7, description='month')
all_months=widgets.Checkbox( value=True, description='all_months')
indicator=widgets.Dropdown(options=list(df_2.select_dtypes('number').columns), value='profit',
                           description='indicator')                   
ui = widgets.HBox(
        [widgets.VBox([year,month]),
        analysis_by,
        widgets.VBox([indicator,all_months])
        ], 
#         layout=Layout(display='flex', flex_flow='row wrap', justify_content='space-between')
    )
def bar_plot(analysis_by,year,month,all_months,indicator):
    if (month not in df_report_2.loc[str(year)].month.to_list()) & (all_months==False):
        return print( 'Sorry, but no data for this month.')
    if all_months:
            df_filter = str(year)
    else:
            df_filter = str(year)+'/'+str(month)
    if analysis_by=='days':
        print('Mean value :', 
            round(df_.set_index('date').loc[df_filter].reset_index().groupby(
            by=df_.set_index('date').loc[df_filter].reset_index()['date'].dt.year).mean()[indicator].iloc[0]))
        return px.bar(df_.set_index('date').fillna(0).loc[df_filter], y=indicator,text=indicator).update_traces(
            texttemplate='%{text:,}').show()
    elif analysis_by=='month':
        print('Mean value :', 
        round(df_report_2.loc[df_filter].groupby(by=df_report_2.loc[df_filter]['year']).mean()[indicator].iloc[0]))
        return px.bar(df_report_2.loc[df_filter], y=indicator,text=indicator).update_traces(
            texttemplate='%{text:,}').show()
out = widgets.interactive_output(bar_plot, {'analysis_by': analysis_by, 'year' : year, 'month': month, 
                                           'all_months': all_months, 'indicator': indicator}) 


# display(ui, out)

#### Charts and widgets are not showing up in github. Here is a picture of the result:
  (to see the interactivity, please run the code locally)

![jupyter](https://github.com/Lirinawi/Interactive_chart/blob/main/result_chart.png?raw=true)